In [5]:
import numpy as np

Test how to isolate notes and chords of song. Also, how to combine tracks into single track.

In [6]:
midifile = '../Jazz_Music_Midi/BeautifulGirl.mid'

In [7]:
from music21 import *

In [8]:
midi_data = converter.parse(midifile)

In [9]:
len(midi_data)

2

In [10]:
melody_stream = midi_data.parts[0]

In [11]:
melody_stream.show('midi')

In [12]:
# Combine voices
melody = stream.Voice()
for m in melody_stream.getElementsByClass(stream.Voice):
    for j in m:
        melody.insert(j.offset, j)

In [13]:
melody.show('midi')

In [21]:
def printNotes(data):
    # Print notes and rests
    print "%s,%s,%s,%s" % ("Note/Chord", "Midi", "Len","Offset")

    notes = [i for i in data.notes]
    for n in notes:
        if isinstance(n, note.Note):
            # alternative n.pitch.nameWithOctave
            print "%s,%s,%s,%s" % (str(n.pitch.name), n.midi, n.quarterLength, n.offset)
        elif isinstance(n, chord.Chord):
            print "%s,%s,%s,%s" % (' '.join([str(x.name) for x in n.pitches]), ' '.join([str(x.midi) for x in n]), \
                                   n.quarterLength, n.offset)

In [16]:
s = instrument.partitionByInstrument(midi_data)

In [17]:
for i in s.parts:
    print i

<music21.stream.Part Piano>


In [18]:
s.show('midi')

In [23]:
s.show('text')

In [22]:
printNotes(midi_data.flat)

Note/Chord,Midi,Len,Offset
G,55,4/3,0.0
B-,58,1.25,2/3
D,62,1.0,1.25
G,67,1.25,7/3
G,55,4/3,2.5
B-,70,1.25,3.0
B-,58,0.75,3.0
D,74,2/3,11/3
D,62,1.0,11/3
F#,66,2.0,4.25
B-,58,1.25,13/3
F#,54,1.75,5.0
D,62,0.75,6.25
B-,58,0.5,6.75
G,79,1.0,22/3
F#,54,1.0,7.5
B-,58,2/3,8.0
D,74,0.75,8.25
D,62,2/3,26/3
B-,70,0.75,9.0
B-,58,1.25,28/3
F,65,1.75,9.75
F,53,2.0,10.0
D,62,1.0,11.25
B-,58,0.25,11.75
F,53,0.75,12.5
G,67,1.0,13.0
B-,58,0.0,13.0
D C,62 72,0.5,13.5
D,74,2/3,14.25
B-,58,0.5,14.25
G,79,1.25,14.75
E,52,0.75,14.75
D,74,1.25,46/3
B-,58,1.0,46/3
D,62,2/3,16.0
C,72,2/3,16.5
B-,58,0.5,50/3
E,52,0.75,17.25
B-,70,0.75,17.75
B-,58,0.75,18.0
C,60,0.5,19.25
C,72,1.5,19.25
F#,66,5/3,19.25
A,45,1.25,19.25
D,38,7/3,19.75
D,50,4/3,21.0
F#,54,0.75,65/3
A,57,1/3,22.25
C,72,0.25,22.75
F#,66,0.0,23.5
F#,66,2/3,24.0
A C,69 60,1.25,24.0
E,64,1.25,24.25
A,45,1.25,24.25
D,38,1.0,24.75
C,60,0.0,26.0
F#,66,0.5,26.0
A,69,2/3,26.0
E,64,0.75,26.0
D,50,2/3,26.0
F#,54,2/3,80/3
A,57,0.75,82/3
B- F#,70 66,0.0,27.75


In [24]:
midi_data.flat.show('midi')

Translate to state matrix. We aren't using that; it's too hard to convert statematrix back using music21.

In [26]:
bpm = 0
for i in midi_data.flat.getElementsByClass(tempo.MetronomeMark):
    bpm = i.number
print bpm

121.0


In [27]:
resolution = 40

quarterDuration = 60/bpm * resolution
print quarterDuration

19.8347107438


In [34]:
def my_round(x):
    return round(x*4)/4

In [35]:
c = chord.Chord([64, 67], quarterLength=my_round(3/quarterDuration))
offset=my_round(8/quarterDuration)

In [36]:
print c.quarterLength

0.25


In [37]:
offset

0.5

In [39]:
timeleft = int(midi_data.flat.duration.quarterLength * quarterDuration)

In [40]:
timeleft

8950

In [41]:
span = 128
statematrix = np.zeros((span * 2, timeleft + 1))

In [43]:
for n in midi_data.flat.notes:
    if isinstance(n, note.Note):
        tick = int(n.offset * quarterDuration)
        length = int(n.quarterLength * quarterDuration)
        statematrix[n.midi, tick] = 1
        statematrix[n.midi + span, tick+length] = 1
    elif isinstance(n, chord.Chord):
        tick = int(n.offset * quarterDuration)
        length = int(n.quarterLength * quarterDuration)
        for m in n:
            statematrix[m.midi, tick] = 1
            statematrix[m.midi + span, tick + length] = 1

In [44]:
statematrix.shape

(256, 8951)